In [1]:
from sentence_transformers import SentenceTransformer, util
from datasets import load_dataset, Dataset
from tqdm.auto import tqdm
import json
import pandas as pd
import numpy as np
from transformers import T5ForConditionalGeneration, T5TokenizerFast 

In [18]:
model = T5ForConditionalGeneration.from_pretrained("BlackKakapo/t5-base-paraphrase-ro")
tokenizer = T5TokenizerFast.from_pretrained("BlackKakapo/t5-base-paraphrase-ro")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [3]:
with open('ds_roitd_ollama_final_sorted_2.json', 'r') as f:
    data = json.load(f)
    
df = pd.DataFrame(data)
df.head(10)

,id,context,question,answers
0,YJ95VF1770,Aplicatii & notificari (disponibil cand telefo...,Ce tip de ecran conduce la o încărcare mai rar...,"{'text': ['Ecran always-on', 'Ecran always-o']..."
1,YL83VU1211,Utilizati senzorul Pulse Ox pentru a verifica ...,Ce senzor este utilizat pentru observarea temp...,"{'text': ['Pulse Ox', 'senzorul Pulse Ox '], '..."
2,DA68SX6997,"Poate fi scufundat pana la 50 de metri, de ase...",Câte moduri de exerciții profesioniste oferă d...,{'text': ['Amazfit GTR accepta 12 moduri de ex...
3,TP61GG6784,Combina frigoriﬁca independenta poate ﬁ acum i...,Ce sistem de închidere a ușilor are dispzoitivul?,"{'text': ['la 90º ', 'Sistemul de deschidere a..."
4,ZJ97KL5342,Frigiderele de sine statatoare de la Miele cu ...,Ce mecanism face mai ușoară închiderea ușilor?,"{'text': ['Click2open', 'Mecanismul Click2open..."
5,AF88XY3440,Sertarul Crispzone.Sertar mare pentru fructe s...,Care este denumirea sertarului care permite sc...,"{'text': ['CrispZone', 'Sertarul Crispzone'], ..."
6,PE47WS5286,DoorCooling+™.Racire cu pana la 32%* mai rapid...,Care este rolul aerisirilor din partea laterală?,{'text': ['mentin temperatura pentru a pastra ...
7,GZ22OI3120,"WiFi: Control de la distanta, prin WiFi, cu aj...",Care este temperatura maximă exterioară la car...,"{'text': ['-15°C', '-15oC'], 'answer_start': [..."
8,ES30DU9549,"Functia PacePro, prima de acest fel, va ajuta ...",Care este succesorul prmei funcții de menținer...,"{'text': ['PacePro', 'Functia PacePro'], 'answ..."
9,HL57XJ9615,Aplicatia de fitness din ceas va permite sa ur...,De unde este accesibilă aplicația meteo?,{'text': ['de pe ecranul de pornire al ceasulu...


In [4]:
eval_answer = SentenceTransformer("BlackKakapo/stsb-xlm-r-multilingual-ro")

In [5]:
print(df.shape)
print(len(data))

(4417, 4)
4417


In [6]:
scores = []
for i in tqdm(range(len(data))):
    try:
        scores.append(util.pytorch_cos_sim(eval_answer.encode(df['answers'][i]['text'][0], convert_to_tensor=True), 
                                       eval_answer.encode(df['question'][i], convert_to_tensor=True)).item())
    except:
        print(type(df['answers'][i]))
        print(df.iloc[i])
        break

  0%|          | 0/4417 [00:00<?, ?it/s]

In [7]:
scores = np.array(scores)
print(scores.mean())
print(scores.std())

0.3584697827005635
0.1850522423436764


In [8]:
scores_filter = scores[(scores > scores.mean() + 2 * scores.std()) | (scores < scores.mean() - 2 * scores.std())]
print(scores_filter.shape)

(115,)


In [20]:
final_data = []
results = {
    0.85: 0,
    0.9: 0
}
good = 0

for i in tqdm(range(len(data))):
    final_data.append(data[i])
    
    if scores[i] not in scores_filter:
        continue
    
    encq = tokenizer.encode_plus(df['question'][i], return_tensors="pt", truncation=True, padding="max_length")
    input_ids, attn_masks = encq["input_ids"], encq["attention_mask"]
    
    outs85 = []
    scores85 = []
    outs9 = []
    scores9 = []

    generated_rephrasings = model.generate(
        input_ids=input_ids,
        attention_mask=attn_masks,
        max_length=256,
        early_stopping=True,
        top_p=0.85,
        top_k=30,
        num_return_sequences=5,
        num_beams=5,
        do_sample=True
    )
    
    for generated in generated_rephrasings:
        tpara = tokenizer.decode(generated, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        
        if df['question'][i].lower() != tpara.lower():
            outs85.append(tpara)
    
    
    generated_rephrasings = model.generate(
        input_ids=input_ids,
        attention_mask=attn_masks,
        max_length=256,
        early_stopping=True,
        top_p=0.9,
        top_k=30,
        num_return_sequences=5,
        num_beams=5,
        do_sample=True
        
    )
    
    for generated in generated_rephrasings:
        tpara = tokenizer.decode(generated, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        
        if df['question'][i].lower() != tpara.lower():
            outs9.append(tpara)
        
    
    for out in outs85:
        scores85.append(util.pytorch_cos_sim(eval_answer.encode(df['answers'][i]['text'][0], convert_to_tensor=True), 
                                       eval_answer.encode(out, convert_to_tensor=True)).item())
        
    for out in outs9:
        scores9.append(util.pytorch_cos_sim(eval_answer.encode(df['answers'][i]['text'][0], convert_to_tensor=True), 
                                       eval_answer.encode(out, convert_to_tensor=True)).item())
    
    max85 = max(scores85)
    max9 = max(scores9)
    
    newq = ''
    if max85 > max9 and max85 < scores.mean() + 2 * scores.std() and max85 > scores.mean() - 2 * scores.std():
        results[0.85] += 1
        for j in range(len(scores85)):
            if scores85[j] == max85:
                newq = outs85[j]
                break
    elif max9 < scores.mean() + 2 * scores.std() and max9 > scores.mean() - 2 * scores.std():
        results[0.9] += 1
        for j in range(len(scores9)):
            if scores9[j] == max9:
                newq = outs9[j]
                break
    
    if newq != '':
        new_data = data[i].copy()
        print(new_data['question'])
        new_data['question'] = newq
        print('----')
        print(newq)
        
        final_data.append(new_data)
        
        good += 1

  0%|          | 0/4417 [00:00<?, ?it/s]

Care este rolul aerisirilor din partea laterală?
----
Care este rolul respirației din partea următoare?
Care este Rezolutia care  iti permite sa apropii imaginea oricat ai nevoie?
----
Care este Rezolutia care permite sa apropia imaginea oricata pe care o doriți?
Câte zile de utilizare sunt permise printr-o încărcare de 1 oră?
----
Câte zile pot folosi într-o încărcare de o oră?
Cât timp poate fi ținută ușa închisă fără a afecta calitatea alimentelor?
----
Cât de mult timp va fi ținută ușa închisă fără să afecteze calitatea alimentelor?
Care este cel mai subțire ceas inteligent de pe piață?
----
Care este cel mai subțire ceasul inteligent din lume?
Câte programe scurte are mașina?
----
Câte programe are această mașină?
Care sunt elementele care nu pot fi afișate pe ecranul LED?
----
Elementele care nu pot fi afișate pe ecranul LED?
Care este cel mai scurt program al mașinii?
----
Care este cel mai scurt program de mașină?
Câte programe scurte are mașina?
----
Câte programe are această 

In [21]:
print(len(data))
print(good)
print(len(final_data))

4417
30
4447


In [23]:
with open('ds_roitd_ollama_sorted_rephrased.json', 'w') as f:
    json.dump(final_data, f, indent=4)